# Experiment 3: Advanced RAG

In [1]:
# Setup
import sys
import json
from pathlib import Path
from typing import Dict, List, Any, Set, Tuple
from dataclasses import dataclass, asdict
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# RAG components
import chromadb
from chromadb.utils import embedding_functions

# LLM
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

sys.path.append('..')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("Imports loaded")

Imports loaded


In [2]:
# Configuration
DB_PATH = Path("../data/vector_db")
MODEL_PATH = Path("/home/sskaplun/study/genAI/kaggle/models/gemma-2-9b-it")
OUTPUT_DIR = Path("../evaluation/experiment_03")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

COLLECTION_NAME = "ukrainian_math"
EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

# Advanced RAG parameters
NUM_QUERY_EXPANSIONS = 3  # Generate 3 query variants
RETRIEVAL_K = 15  # Retrieve more candidates
FINAL_K = 5  # Re-rank to top-5
TEMPERATURE = 0.7
MAX_NEW_TOKENS = 512

# Re-ranking weights
RELEVANCE_WEIGHT = 0.5
DIVERSITY_WEIGHT = 0.3
CONTENT_TYPE_WEIGHT = 0.2

print(f"Query Expansions: {NUM_QUERY_EXPANSIONS}")
print(f"Retrieval K: {RETRIEVAL_K} → Final K: {FINAL_K}")
print(f"CUDA: {torch.cuda.is_available()}")

Query Expansions: 3
Retrieval K: 15 → Final K: 5
CUDA: True


In [3]:
@dataclass
class RetrievedChunk:
    text: str
    content_type: str
    confidence: float
    filename: str
    page_start: int
    page_end: int
    distance: float
    relevance: float
    rerank_score: float  # NEW: re-ranking score
    citation: str

@dataclass
class AdvancedRAGResponse:
    question: str
    expanded_queries: List[str]  # NEW: query variants
    answer: str
    citations: List[str]
    retrieved_chunks: List[RetrievedChunk]
    avg_relevance: float
    avg_rerank_score: float  # NEW
    answer_length: int
    
    def to_dict(self):
        return {
            'question': self.question,
            'expanded_queries': self.expanded_queries,
            'answer': self.answer,
            'citations': self.citations,
            'avg_relevance': self.avg_relevance,
            'avg_rerank_score': self.avg_rerank_score,
            'answer_length': self.answer_length,
            'num_chunks': len(self.retrieved_chunks)
        }

print("Dataclasses defined")

Dataclasses defined


## 1. Load Vector Database

In [4]:
print("="*80)
print("LOADING VECTOR DATABASE")
print("="*80)

client = chromadb.PersistentClient(path=str(DB_PATH))

embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBEDDING_MODEL
)

collection = client.get_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_function
)

print(f"\nCollection: {COLLECTION_NAME}")
print(f"  Total chunks: {collection.count():,}")

LOADING VECTOR DATABASE



Collection: ukrainian_math
  Total chunks: 15,836


## 2. Load LLM

In [5]:
print("="*80)
print("LOADING LLM")
print("="*80)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(str(MODEL_PATH))
model = AutoModelForCausalLM.from_pretrained(
    str(MODEL_PATH),
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)

print("Model loaded")

LOADING LLM


The tokenizer you are loading from '/home/sskaplun/study/genAI/kaggle/models/gemma-2-9b-it' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded


## 3. Query Expansion

In [6]:
def expand_query(query: str, num_variants: int = NUM_QUERY_EXPANSIONS) -> List[str]:
    """
    Generate multiple query variants for better retrieval coverage.
    
    Strategy:
    1. Original query
    2. Extract key math terms and rephrase
    3. Add context (e.g., "формула", "приклад", "визначення")
    """
    queries = [query]  # Always include original
    
    # Generate variants using simple LLM prompting
    expansion_prompt = f"""Перефразуй це запитання українською мовою {num_variants-1} різними способами, 
зберігаючи математичний зміст. Використовуй різні формулювання та синоніми.

Оригінальне запитання: {query}

Варіанти (по одному на рядок):"""
    
    messages = [{"role": "user", "content": expansion_prompt}]
    formatted = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    result = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    ).strip()
    
    # Parse variants (split by newlines)
    variants = [line.strip() for line in result.split('\n') if line.strip()]
    queries.extend(variants[:(num_variants-1)])
    
    return queries[:num_variants]

print("Query expansion function defined")

Query expansion function defined


## 4. Hybrid Retrieval & Re-ranking

In [7]:
def retrieve_with_queries(
    queries: List[str],
    k: int = RETRIEVAL_K
) -> List[RetrievedChunk]:
    """
    Retrieve chunks using multiple queries and merge results.
    """
    all_chunks = {}
    
    for query in queries:
        results = collection.query(
            query_texts=[query],
            n_results=k
        )
        
        for doc, meta, dist in zip(
            results['documents'][0],
            results['metadatas'][0],
            results['distances'][0]
        ):
            # Use text as key to deduplicate
            key = doc[:100]  # First 100 chars as key
            
            if key not in all_chunks:
                chunk = RetrievedChunk(
                    text=doc,
                    content_type=meta['content_type'],
                    confidence=meta['confidence'],
                    filename=meta['filename'],
                    page_start=meta['page_start'],
                    page_end=meta['page_end'],
                    distance=dist,
                    relevance=1 - dist,
                    rerank_score=0.0,  # Will be computed later
                    citation=f"[{meta['filename']}, с. {meta['page_start']}-{meta['page_end']}]"
                )
                all_chunks[key] = chunk
            else:
                # Update with better relevance if found
                if (1 - dist) > all_chunks[key].relevance:
                    all_chunks[key].distance = dist
                    all_chunks[key].relevance = 1 - dist
    
    return list(all_chunks.values())

print("Hybrid retrieval function defined")

Hybrid retrieval function defined


In [8]:
def calculate_diversity_score(chunks: List[RetrievedChunk]) -> List[float]:
    """
    Calculate diversity scores based on content type and source variety.
    """
    # Count occurrences
    type_counts = Counter(c.content_type for c in chunks)
    file_counts = Counter(c.filename for c in chunks)
    
    diversity_scores = []
    for chunk in chunks:
        # Penalize over-represented types and files
        type_penalty = 1.0 / type_counts[chunk.content_type]
        file_penalty = 1.0 / file_counts[chunk.filename]
        diversity_score = (type_penalty + file_penalty) / 2
        diversity_scores.append(diversity_score)
    
    # Normalize to [0, 1]
    max_score = max(diversity_scores) if diversity_scores else 1.0
    return [s / max_score for s in diversity_scores]

def calculate_content_type_score(chunk: RetrievedChunk) -> float:
    """
    Score chunks by content type preference for task generation.
    
    Preference order:
    1. explanation (best for understanding concepts)
    2. definition (good for terminology)
    3. problem (examples of tasks)
    4. example, theorem, etc.
    """
    type_scores = {
        'explanation': 1.0,
        'definition': 0.9,
        'problem': 0.8,
        'example': 0.7,
        'theorem': 0.7,
        'formula': 0.6
    }
    return type_scores.get(chunk.content_type, 0.5)

def rerank_chunks(
    chunks: List[RetrievedChunk],
    final_k: int = FINAL_K
) -> List[RetrievedChunk]:
    """
    Re-rank chunks using weighted combination of:
    - Semantic relevance (from embedding distance)
    - Content diversity (variety of types/sources)
    - Content type preference (explanations > examples)
    """
    diversity_scores = calculate_diversity_score(chunks)
    
    for i, chunk in enumerate(chunks):
        relevance = chunk.relevance
        diversity = diversity_scores[i]
        content_type = calculate_content_type_score(chunk)
        
        # Weighted combination
        rerank_score = (
            RELEVANCE_WEIGHT * relevance +
            DIVERSITY_WEIGHT * diversity +
            CONTENT_TYPE_WEIGHT * content_type
        )
        chunk.rerank_score = rerank_score
    
    # Sort by rerank score and take top-k
    chunks.sort(key=lambda c: c.rerank_score, reverse=True)
    return chunks[:final_k]

print("Re-ranking functions defined")

Re-ranking functions defined


## 5. Advanced RAG Pipeline

In [9]:
SYSTEM_PROMPT = """Ти — досвідчений викладач математики для українських учнів 10-11 класів.

Твоє завдання:
- Згенерувати математичну задачу з розв'язанням на основі ТІЛЬКИ наданого контексту
- Використовувати ТІЛЬКИ українську мову
- Використовувати математичну термінологію з підручників
- Обов'язково посилатися на джерела
- Надати чітке покрокове розв'язання

Формат відповіді:
**Задача:** [текст задачі на основі контексту]

**Розв'язання:**
[покрокове рішення з посиланнями на джерела]

**Відповідь:** [фінальна відповідь]

ВАЖЛИВО: Використовуй ТІЛЬКИ інформацію з наданого контексту!"""

def format_context(chunks: List[RetrievedChunk]) -> str:
    """Format chunks with re-rank scores."""
    context_parts = []
    for i, chunk in enumerate(chunks, 1):
        header = f"[Джерело {i}] {chunk.citation} | Тип: {chunk.content_type} | Оцінка: {chunk.rerank_score:.3f}"
        context_parts.append(f"{header}\n{chunk.text}")
    return "\n\n".join(context_parts)

def generate_answer(prompt: str) -> str:
    """Generate using LLM."""
    messages = [{"role": "user", "content": prompt}]
    formatted = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=TEMPERATURE,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=True
    ).strip()

def advanced_rag_generate(
    question: str,
    verbose: bool = False
) -> AdvancedRAGResponse:
    """
    Advanced RAG pipeline with query expansion and re-ranking.
    """
    if verbose:
        print(f"\nQuestion: {question}")
        print("  Step 1: Query expansion...")
    
    # 1. Query expansion
    expanded_queries = expand_query(question)
    if verbose:
        for i, q in enumerate(expanded_queries, 1):
            print(f"    {i}. {q}")
        print(f"  Step 2: Hybrid retrieval (k={RETRIEVAL_K})...")
    
    # 2. Retrieve with multiple queries
    chunks = retrieve_with_queries(expanded_queries)
    if verbose:
        print(f"    Retrieved {len(chunks)} unique chunks")
        print(f"  Step 3: Re-ranking to top-{FINAL_K}...")
    
    # 3. Re-rank
    reranked_chunks = rerank_chunks(chunks, final_k=FINAL_K)
    if verbose:
        avg_rel = np.mean([c.relevance for c in reranked_chunks])
        avg_rerank = np.mean([c.rerank_score for c in reranked_chunks])
        print(f"    Avg relevance: {avg_rel:.3f} | Avg rerank score: {avg_rerank:.3f}")
        print("  Step 4: Generating answer...")
    
    # 4. Generate
    context = format_context(reranked_chunks)
    prompt = f"{SYSTEM_PROMPT}\n\nКОНТЕКСТ З ПІДРУЧНИКІВ:\n{context}\n\nЗАПИТАННЯ:\n{question}\n\nТВОЯ ВІДПОВІДЬ:"
    answer = generate_answer(prompt)
    
    if verbose:
        print(f"    Generated {len(answer)} chars")
    
    return AdvancedRAGResponse(
        question=question,
        expanded_queries=expanded_queries,
        answer=answer,
        citations=[c.citation for c in reranked_chunks],
        retrieved_chunks=reranked_chunks,
        avg_relevance=float(np.mean([c.relevance for c in reranked_chunks])),
        avg_rerank_score=float(np.mean([c.rerank_score for c in reranked_chunks])),
        answer_length=len(answer)
    )

print("Advanced RAG pipeline defined")

Advanced RAG pipeline defined


## 6. Test Questions

In [10]:
from common import STANDARD_TEST_QUESTIONS, EVALUATION_DATASET

TEST_QUESTIONS = STANDARD_TEST_QUESTIONS
print(f"Test set: {len(TEST_QUESTIONS)} questions")

# Create mapping of questions to expected answers
question_to_expected = {q['input']: q['expected_answer'] for q in EVALUATION_DATASET}
print(f"Expected answers loaded for {len(question_to_expected)} questions")

Test set: 15 questions
Expected answers loaded for 10 questions


## 7. Run Advanced RAG Experiment

In [11]:
print("="*80)
print("RUNNING ADVANCED RAG EXPERIMENT")
print("="*80)

responses = []

for i, question in enumerate(TEST_QUESTIONS, 1):
    print(f"\n[{i}/{len(TEST_QUESTIONS)}] {question}")
    print("-"*80)
    
    response = advanced_rag_generate(question, verbose=True)
    responses.append(response)
    
    print(f"\nAnswer:\n{response.answer}")
    print(f"\nTop-3 Citations:")
    for j, citation in enumerate(response.citations[:3], 1):
        print(f"  {j}. {citation}")

print(f"\n{'='*80}")
print(f"Completed {len(responses)} advanced RAG responses")
print("="*80)

RUNNING ADVANCED RAG EXPERIMENT

[1/15] Квадратні рівняння
--------------------------------------------------------------------------------

Question: Квадратні рівняння
  Step 1: Query expansion...


    1. Квадратні рівняння
    2. ## Перефразування питання "Квадратні рівняння":
    3. 1. **Чи знаєте ви, як розв'язувати рівняння другого степеня?**
  Step 2: Hybrid retrieval (k=15)...
    Retrieved 41 unique chunks
  Step 3: Re-ranking to top-5...
    Avg relevance: 0.781 | Avg rerank score: 0.779
  Step 4: Generating answer...


    Generated 532 chars

Answer:
**Задача:** Розв'яжіть рівняння:  x² - 3x + 2 = 0

**Розв'язання:**

Для розв'язування квадратного рівняння x² - 3x + 2 = 0 можна використовувати формулу для розв'язування квадратних рівнянь:

x = (-b ± √(b² - 4ac)) / 2a

де a, b та c - коефіцієнти рівняння.

У нашому випадку a = 1, b = -3, c = 2. Підставляємо ці значення у формулу:

x = (3 ± √((-3)² - 4 * 1 * 2)) / (2 * 1)

x = (3 ± √(9 - 8)) / 2

x = (3 ± √1) / 2

x = (3 ± 1) / 2

Отже, отримуємо два розв'язки:

x₁ = (3 + 1) / 2 = 2

x₂ = (3 - 1) / 2 = 1

**Відповідь:** 1, 2

Top-3 Citations:
  1. [merzlyak-ag-algebra-i-pochatky-analizu-prof-riven-11-kl.pdf, с. 19-19]
  2. [geometriya-pidruchnyk-dlya-9-klasu-zagalnoosvitnih-navchalnyh-zakladiv-jershova-a-p-goloborodko-v-v-kryzhanovskyy-o-f-jershov-s-v-1.pdf, с. 48-48]
  3. [ister-algeb-p-9ukr-054-13-s.pdf, с. 126-126]

[2/15] Квадратні рівняння
--------------------------------------------------------------------------------

Question: Квадратні рівнян

    1. Квадратні рівняння
    2. ##  Варіанти перефразування:
    3. 1.  **Чи знаєте ви, як розв'язувати рівняння другого ступеня?**
  Step 2: Hybrid retrieval (k=15)...
    Retrieved 44 unique chunks
  Step 3: Re-ranking to top-5...
    Avg relevance: 0.785 | Avg rerank score: 0.772
  Step 4: Generating answer...


    Generated 622 chars

Answer:
**Задача:**  Розв'яжіть рівняння 2x² - 3x = 5x + 1

**Розв'язання:**

1. **Перемістимо всі члени рівняння в одну частину:**
   2x² - 3x - 5x - 1 = 0
2. **Згрупуємо подібні члени:**
   2x² - 8x - 1 = 0
3. **Використовуємо формулу для розв'язування квадратного рівняння:**
   x = (-b ± √(b² - 4ac)) / 2a
   де a = 2, b = -8, c = -1
4. **Підставимо значення:**
   x = (8 ± √((-8)² - 4 * 2 * -1)) / (2 * 2)
   x = (8 ± √(64 + 8)) / 4
   x = (8 ± √72) / 4
   x = (8 ± 6√2) / 4
5. **Зокремлюємо корені:**
   x₁ = (8 + 6√2) / 4 = 2 + (3/2)√2
   x₂ = (8 - 6√2) / 4 = 2 - (3/2)√2

**Відповідь:**  x₁ = 2 + (3/2)√2, x₂ = 2 - (3/2)√2

Top-3 Citations:
  1. [merzlyak-ag-algebra-i-pochatky-analizu-prof-riven-11-kl.pdf, с. 19-19]
  2. [geometriya-pidruchnyk-dlya-9-klasu-zagalnoosvitnih-navchalnyh-zakladiv-jershova-a-p-goloborodko-v-v-kryzhanovskyy-o-f-jershov-s-v-1.pdf, с. 48-48]
  3. [ister-algeb-p-9ukr-054-13-s.pdf, с. 126-126]

[3/15] Квадратні рівняння
------------------

...

    Generated 630 chars

Answer:
**Задача:**  Знайдіть область визначення функції:  f(x) = log(6 - x)

**Розв'язання:**

1. **Область визначення логарифмічної функції:**  Область визначення логарифмічної функції  y = log_a(x)  є сукупністю всіх значень аргументу x, для яких  x > 0.

2. **Визначимо область визначення нашої функції:** Для нашої функції f(x) = log(6 - x) потрібно знайти всі значення x, для яких 6 - x > 0.

3. **Розв'яжемо нерівність:** 
   6 - x > 0
   -x > -6
   x < 6

4. **Запишемо область визначення:**  Область визначення функції f(x) = log(6 - x)  є інтервал (-∞, 6).

**Відповідь:** Область визначення функції f(x) = log(6 - x) є (-∞, 6).

Top-3 Citations:
  1. [merzlyak-a-g-algebra-i-poch-analizu-pochatok-vyvch-na-poglyb-rivni-z-8-kl-prof-riven-11-kl.pdf, с. 36-36]
  2. [merzlyak-ag-matematyka-algebra-i-poch-analizu-ta-geometriya-riven-standartu-11-kl.pdf, с. 24-24]
  3. [merzlyak-ag-matematyka-algebra-i-poch-analizu-ta-geometriya-riven-standartu-11-kl.pdf, с. 32-32]


    1. Логарифми
    2. ## Перефразировані запитання про логарифми:
    3. 1. **Які значення числа є аргументами логарифма, якщо результат дорівнює заданій величині?**
  Step 2: Hybrid retrieval (k=15)...
    Retrieved 39 unique chunks
  Step 3: Re-ranking to top-5...
    Avg relevance: 0.794 | Avg rerank score: 0.767
  Step 4: Generating answer...


    Generated 322 chars

Answer:
**Задача:**  Розв'яжіть рівняння: 2<sup>x</sup> = 16

**Розв'язання:**

1.  Знайдемо логарифм обох сторін рівняння за основою 2: 
    log₂(2<sup>x</sup>) = log₂16

2.  Застосуємо властивість логарифму степеня: log₂(2<sup>x</sup>) = x

3.  Знайдемо логарифм 16 за основою 2: log₂16 = 4

4.  Тоді x = 4

**Відповідь:** x = 4

Top-3 Citations:
  1. [merzlyak-ag-matematyka-algebra-i-poch-analizu-ta-geometriya-riven-standartu-11-kl.pdf, с. 25-25]
  2. [merzlyak-a-g-algebra-i-poch-analizu-pochatok-vyvch-na-poglyb-rivni-z-8-kl-prof-riven-11-kl.pdf, с. 35-35]
  3. [merzlyak-ag-algebra-i-pochatky-analizu-prof-riven-11-kl.pdf, с. 82-82]

[15/15] Логарифми
--------------------------------------------------------------------------------

Question: Логарифми
  Step 1: Query expansion...


    1. Логарифми
    2. 1.  **Який результат отримуємо, коли знаходимо логарифм певного числа відносно заданої основи?**
    3. 2.  **Що таке логарифм числа  за певною основою?**
  Step 2: Hybrid retrieval (k=15)...
    Retrieved 36 unique chunks
  Step 3: Re-ranking to top-5...
    Avg relevance: 0.774 | Avg rerank score: 0.745
  Step 4: Generating answer...


    Generated 523 chars

Answer:
**Задача:**

Знайдіть значення виразу: log 5 ⋅ log 10 ⋅ log 25

**Розв'язання:**

*  Використовуємо властивість логарифмів: log a<sup>b</sup> = b ⋅ log a
    * log 25 = log 5<sup>2</sup> = 2 ⋅ log 5

* Підставляємо отримане значення в вираз:
    * log 5 ⋅ log 10 ⋅ log 25 = log 5 ⋅ log 10 ⋅ (2 ⋅ log 5)

* Згадуємо, що log 10 = 1
    * log 5 ⋅ log 10 ⋅ (2 ⋅ log 5) = log 5 ⋅ 1 ⋅ (2 ⋅ log 5) = 2 ⋅ (log 5)<sup>2</sup>

* Обчислюємо: 2 ⋅ (log 5)<sup>2</sup> ≈ 2 ⋅ 0.699 ≈ 1.398

**Відповідь:** log 5 ⋅ log 10 ⋅ log 25 ≈ 1.398

Top-3 Citations:
  1. [merzlyak-ag-matematyka-algebra-i-poch-analizu-ta-geometriya-riven-standartu-11-kl.pdf, с. 24-24]
  2. [matematyka-11-kl-bevz.pdf, с. 24-24]
  3. [alhebra-i-pochatky-analizu-profilnyi-riven-pidruchnyk-dlia-11-klasu-zzso-avt-nelin-ye-p-dolgova-o-ye.pdf, с. 38-38]

Completed 15 advanced RAG responses


## 8. Evaluation

In [12]:
import common

print("Evaluation functions loaded from common.py")

Evaluation functions loaded from common.py


In [13]:
# Evaluate
print("="*80)
print("EVALUATION")
print("="*80)

evaluations = []

for i, response in enumerate(responses, 1):
    expected_answer = question_to_expected.get(response.question, None)
    metrics = common.evaluate_advanced_rag(
        response.answer, 
        response.answer_length, 
        response.avg_relevance, 
        response.avg_rerank_score,
        expected_answer
    )
    evaluations.append({
        'question': response.question,
        'metrics': metrics,
        'answer_length': response.answer_length,
        'avg_relevance': response.avg_relevance,
        'avg_rerank': response.avg_rerank_score
    })
    
    print(f"\n{i}. {response.question[:50]}...")
    print(f"   Overall: {metrics['overall_score']:.3f} | "
          f"Rerank: {metrics['rerank_quality']:.3f} | "
          f"Ukrainian: {metrics['ukrainian_ratio']:.3f}")

# Summary
print(f"\n{'='*80}")
print("SUMMARY")
print("="*80)

avg_metrics = {
    'overall_score': np.mean([e['metrics']['overall_score'] for e in evaluations]),
    'retrieval_quality': np.mean([e['metrics']['retrieval_quality'] for e in evaluations]),
    'rerank_quality': np.mean([e['metrics']['rerank_quality'] for e in evaluations]),
    'ukrainian_ratio': np.mean([e['metrics']['ukrainian_ratio'] for e in evaluations]),
    'completeness': np.mean([e['metrics']['completeness'] for e in evaluations]),
    'correctness': np.mean([e['metrics']['correctness'] for e in evaluations]),
    'structure_rate': sum(e['metrics']['has_structure'] for e in evaluations) / len(evaluations),
    'citation_rate': sum(e['metrics']['has_citations'] for e in evaluations) / len(evaluations)
}

for key, value in avg_metrics.items():
    print(f"  {key:20s}: {value:.3f}")


EVALUATION

1. Квадратні рівняння...
   Overall: 0.762 | Rerank: 0.779 | Ukrainian: 0.911

2. Квадратні рівняння...
   Overall: 0.905 | Rerank: 0.772 | Ukrainian: 0.886

3. Квадратні рівняння...
   Overall: 0.775 | Rerank: 0.754 | Ukrainian: 0.995

4. Теорема Піфагора...
   Overall: 0.761 | Rerank: 0.748 | Ukrainian: 0.942

5. Теорема Піфагора...
   Overall: 0.765 | Rerank: 0.758 | Ukrainian: 0.917

6. Теорема Піфагора...
   Overall: 0.732 | Rerank: 0.779 | Ukrainian: 0.819

7. Площа трикутника...
   Overall: 0.774 | Rerank: 0.867 | Ukrainian: 0.870

8. Площа трикутника...
   Overall: 0.786 | Rerank: 0.876 | Ukrainian: 0.911

9. Площа трикутника...
   Overall: 0.803 | Rerank: 0.865 | Ukrainian: 0.980

10. Об'єм кулі...
   Overall: 0.645 | Rerank: 0.713 | Ukrainian: 0.943

11. Об'єм кулі...
   Overall: 0.767 | Rerank: 0.754 | Ukrainian: 0.993

12. Об'єм кулі...
   Overall: 0.765 | Rerank: 0.781 | Ukrainian: 0.973

13. Логарифми...
   Overall: 0.736 | Rerank: 0.682 | Ukrainian: 0.906

14

## 9. Save Results

In [14]:
results = {
    'experiment': 'advanced_rag',
    'description': 'Query expansion + hybrid retrieval + re-ranking',
    'model': 'gemma-2-9b-it',
    'config': {
        'query_expansions': NUM_QUERY_EXPANSIONS,
        'retrieval_k': RETRIEVAL_K,
        'final_k': FINAL_K,
        'relevance_weight': RELEVANCE_WEIGHT,
        'diversity_weight': DIVERSITY_WEIGHT,
        'content_type_weight': CONTENT_TYPE_WEIGHT
    },
    'avg_metrics': avg_metrics,
    'responses': [r.to_dict() for r in responses],
    'evaluations': evaluations
}

with open(OUTPUT_DIR / 'results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"Results saved to {OUTPUT_DIR}")
print("\n" + "="*80)
print("EXPERIMENT 3 COMPLETE")
print("="*80)
print(f"\nOverall Score: {avg_metrics['overall_score']:.3f}")
print(f"Retrieval Quality: {avg_metrics['retrieval_quality']:.3f}")
print(f"Re-rank Quality: {avg_metrics['rerank_quality']:.3f}")
print(f"Ukrainian Ratio: {avg_metrics['ukrainian_ratio']:.3f}")

Results saved to ../evaluation/experiment_03

EXPERIMENT 3 COMPLETE

Overall Score: 0.756
Retrieval Quality: 0.769
Re-rank Quality: 0.776
Ukrainian Ratio: 0.895
